In [205]:
import json, os, pickle, random
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn

import qiskit
from qiskit import QuantumCircuit, execute
from qiskit.compiler import transpile
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.circuit.library import CXGate, RXGate, IGate, ZGate
from qiskit.providers.fake_provider import FakeMontreal, FakeLima

from utils import get_backend_properties_v1

# from blackwater.data.utils import (
#     generate_random_pauli_sum_op,
#     create_estimator_meas_data,
#     circuit_to_graph_data_json,
#     get_backend_properties_v1,
#     encode_pauli_sum_op,
#     create_meas_data_from_estimators
# )

from mlp import MLP1, MLP2, MLP3, encode_data

#from mbd_utils import cal_z_exp, generate_disorder, construct_mbl_circuit, calc_imbalance, modify_and_add_noise_to_model

import matplotlib.pyplot as plt
import seaborn as sns
from noise_utils import AddNoise, RemoveReadoutErrors

In [206]:
from mlp import recursive_dict_loop, count_gates_by_rotation_angle

In [207]:
from mbd_utils import calc_imbalance
import torch
from torch import nn
from sklearn import datasets
import sklearn

In [208]:
def fix_random_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    print(f'random seed fixed to {seed}')

In [209]:
backend = FakeLima()
properties = get_backend_properties_v1(backend)

## Local
backend_ideal = QasmSimulator() # Noiseless
backend_noisy = AerSimulator.from_backend(FakeLima()) # Noisy

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}
run_config_noisy = {'shots': 10000, 'backend': backend_noisy, 'name': 'noisy'}

In [210]:
backend = FakeLima()
properties = get_backend_properties_v1(backend)

# Local, coherent noise
backend_ideal = QasmSimulator() # Noiseless
backend_noisy_coherent, noise_model = AddNoise(backend=backend).add_coherent_noise(seed=0, theta=np.pi * 0.04, uniform=False, add_depolarization=True)

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}
run_config_noisy_coherent = {'shots': 10000, 'backend': backend_noisy_coherent, 'name': 'noisy_coherent'}

random seed fixed to 0
thetas [0.06896594 0.08987335 0.07574548 0.06847204 0.05323803 0.08116545
 0.05498883 0.1120635 ]


In [211]:
backend = FakeLima()
properties = get_backend_properties_v1(backend)

# Local, coherent noise
backend_ideal = QasmSimulator() # Noiseless
backend_noisy_no_readout = RemoveReadoutErrors().remove_readout_errors()[0]

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}

# no measurement errors 
run_config_noisy_no_readout = {'shots': 10000, 'backend': backend_noisy_no_readout, 'name': 'noisy_no_readout'}

num_qubit = 4

In [212]:
from qiskit.quantum_info import DensityMatrix, partial_trace
from qiskit import IBMQ, pulse
from qiskit.circuit.library import QFT

In [213]:
def load_circuits(data_dir, f_ext='.json'):
    circuits = []
    ideal_exp_vals = []
    noisy_exp_vals = []
    data_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(f_ext)]
    for data_file in tqdm(data_files[1:], leave=True):
        #print(data_file)
        if f_ext == '.json':
            for entry in json.load(open(data_file, 'r')):
                circuits.append(QuantumCircuit.from_qasm_str(entry['circuit']))
                ideal_exp_vals.append(entry['ideal_exp_value'])
                noisy_exp_vals.append(entry['noisy_exp_values'])
        elif f_ext == '.pk':
            for entry in pickle.load(open(data_file, 'rb')):
                circuits.append(entry['circuit'])
                ideal_exp_vals.append(entry['ideal_exp_value'])
                noisy_exp_vals.append(entry['noisy_exp_values'])
    return circuits, ideal_exp_vals, noisy_exp_vals

In [214]:
train_circuits, train_ideal_exp_vals, train_noisy_exp_vals_input = load_circuits('./data/ising_init_from_qasm_no_readout/train/', '.pk')
print(len(train_circuits))

test_circuits, test_ideal_exp_vals, test_noisy_exp_vals_input = load_circuits('./data/ising_init_from_qasm_no_readout/val/', '.pk')
print(len(test_circuits))

  0%|          | 0/14 [00:00<?, ?it/s]

4200


  0%|          | 0/14 [00:00<?, ?it/s]

1400


topology circuit from dag method
<!-- total n of gates
n of cnots
backend properties -->
try training on mukltiple backends
use model noise instead of backend (there is some link between model noise and the backends)
maybe there is something that can be learned from how the noise is modelled some property
use zne to get "worse" noise and let the model learn from it (so maybe it learns some features)



In [215]:
gate_dict = {'nothing' : 0}

In [216]:
# python refuses to cooperate :D
def get_dict_max(dict):
    max = -1
    for item in dict:
        if dict[item] > max:
            max = dict[item]
    return max

In [217]:
def remove_str_in_data(data):
    return [item for item in data if not isinstance(item, str)]

In [218]:
def unpack_data(data):
    flattened_list = []
    for node_type, node_data in data['nodes'].items():
        for sublist in node_data:
            flattened_list.extend(sublist)
    for node_type, node_data in data['edges'].items():
        for sublist in node_data:
            flattened_list.extend(sublist)
    return flattened_list

In [219]:
from utils import circuit_to_graph_data_json

In [220]:
def encode_data_circuit(circuits, ideal_exp_vals, noisy_exp_vals):
    X = []
    y = []
    i = 0
    for quantum_circuit in circuits:
        features = []
        # converts qc.data into list of operations
        for instr, qargs, _ in quantum_circuit.data:
            gate_name = instr.name
            
            if not (gate_name in gate_dict):
                gate_dict[gate_name] = get_dict_max(gate_dict) + 1
                gate_encoding = gate_dict[gate_name]
            else:
                gate_encoding = gate_dict[gate_name]

            if gate_name == 'barrier':
                break

            gate_params = instr.params if instr.params else []
            qubit_indices = [qarg.index for qarg in qargs]

            feature = []

            feature.append(gate_encoding)
            if type(gate_params) == list:
                for param in gate_params:
                    feature.append(float(param))
            else:
                feature.append(float(gate_params))
            feature.extend(qubit_indices)

            if len(feature) == 2:
                feature = [feature[0], 0, feature[1]]

            for item in feature: features.append (item)

        while (len(features)) < 39:
            features.append(0)

        features.append(noisy_exp_vals[i][0][0])

        i += 1

        graph_data = circuit_to_graph_data_json(
            circuit=quantum_circuit,
            properties=properties,
            use_qubit_features=True,
            use_gate_features=True,
        )

        resulting_graph_features = remove_str_in_data(unpack_data(graph_data))

        for item in resulting_graph_features: features.append(float(item))

        while (len(features)) < 14000:
            features.append(0)

        X.append(features)

        #print(X)

    for val in ideal_exp_vals:
        y.append(val[0])

    X = torch.FloatTensor(X)
    y = torch.FloatTensor(y)

    return X, y


In [221]:
X_train, y_train = encode_data_circuit(train_circuits, train_ideal_exp_vals, train_noisy_exp_vals_input)
X_test, y_test = encode_data_circuit(test_circuits, test_ideal_exp_vals, test_noisy_exp_vals_input)

In [222]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import root_mean_squared_error
rfr = RandomForestRegressor(n_estimators=300)
rfr.fit(X_train, y_train)

RandomForestRegressor(n_estimators=300)

In [223]:
y_pred_rf = rfr.predict(X_test)
rms = root_mean_squared_error(y_test, y_pred_rf)

In [224]:
print(rms)

0.020062797006060826
